<a href="https://colab.research.google.com/github/yusufemrekatkat/E-Commerce-Analytics-Basic_Projects/blob/main/Pricing_Optimization1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Customer Behavior & Pricing Analysis (Price Elasticity)

# If we lower the price by X%, we increase demand by Y%, which maximizes revenue at this point.

Price Elasticity of Demand


$$E_d = \frac{\% \Delta Q}{\% \Delta P}$$

Income function:

$R(P) = P \times Q(P)$


Q(P) is demand function dependent on price.

The goal is finding P that maximize the Income; to set the derivative equal to zero:

$$\frac{dR}{dP} = 0$$



**1: Synthetic Data Generation (Linear Demand Curve)**


As prices rise, demand will decrease, but there will be some "noise" in the data because human behavior isn't 100% rational.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# 1. DATA GENERATION (Demand Curve Simulation)
np.random.seed(42)

def generate_sales_data(n_days=365):
    # Randomly generate prices (e.g., varying between $50 and $200)
    prices = np.random.uniform(50, 200, n_days)
    
    # Demand Function (Q = a - b*P + noise)
    # Assumption: When Price is 0, demand is 1000 (Intercept)
    # Slope: Demand drops by 3.5 units for every $1 increase in price.
    noise = np.random.normal(0, 50, n_days) # Add randomness (noise)
    quantities = 1000 - (3.5 * prices) + noise
    
    # Sales cannot be negative, set to 0
    quantities = np.clip(quantities, 0, None)
    
    data = pd.DataFrame({'Price': prices, 'Quantity_Sold': quantities})
    data['Total_Revenue'] = data['Price'] * data['Quantity_Sold']
    
    return data

df = generate_sales_data()

# View Data
print(df.head())

# Plot Distribution (Demand Curve)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Price', y='Quantity_Sold', data=df, color='blue', alpha=0.6)
plt.title('Demand Curve: As price increases, demand decreases')
plt.xlabel('Price ($)')
plt.ylabel('Quantity Sold')
plt.show()

**2: Training and Calculating Elasticity**


We will derive the demand curve using **Linear Regression**.

Curve equation: $Q = a - bP$

This equation will tell us how price-sensitive the customer is.

In [ ]:
# 2. MODELING (Deriving the Demand Function)

X = df[['Price']] # Independent Variable
y = df['Quantity_Sold'] # Dependent Variable

model = LinearRegression()
model.fit(X, y)

# Equation Coefficients
intercept = model.intercept_ # Intercept (a)
slope = model.coef_[0] # Slope (b) - Price Sensitivity

print(f"Demand Function: Q = {intercept:.2f} + ({slope:.2f} * P)")
print(f"Insight: If we increase the price by $1, sales drop by {abs(slope):.2f} units on average.")

**3: Optimization (Revenue Maximization)**

As a PM, we are answering a critical question: "**Which price** point will maximize the revenue?"

In [ ]:
# 3. OPTIMIZATION (Finding the Optimal Price)

# Test all possible prices (Test Range)
price_range = np.linspace(10, 300, 500).reshape(-1, 1)

# Get predicted sales quantities for these prices from the model
predicted_quantities = model.predict(price_range)

# Calculate Revenue (P * Q)
predicted_revenues = price_range.flatten() * predicted_quantities

# Find maximum revenue
max_revenue_index = np.argmax(predicted_revenues)
optimal_price = price_range[max_revenue_index][0]
max_revenue = predicted_revenues[max_revenue_index]

print(f"------------------------------------------------")
print(f"OPTIMAL PRICE: ${optimal_price:.2f}")
print(f"ESTIMATED MAXIMUM REVENUE: ${max_revenue:.2f}")
print(f"------------------------------------------------")

# Chart: Price vs. Revenue (Inverted U Curve - Parabola)
plt.figure(figsize=(10, 6))
plt.plot(price_range, predicted_revenues, color='green', linewidth=2)
plt.axvline(optimal_price, color='red', linestyle='--', label=f'Optimal Price: ${optimal_price:.2f}')
plt.title('Revenue Maximization')
plt.xlabel('Price ($)')
plt.ylabel('Estimated Total Revenue($)')
plt.legend()
plt.grid(True)
plt.show()